<a href="https://colab.research.google.com/github/DataParadox/iTrustBD/blob/main/Codes/iTrustBD_SIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/PG/CSE6413 Network Science/Project Works/'

/content/drive/My Drive/PG/CSE6413 Network Science/Project Works


# Set Datasets

In [ ]:
def setDatasets(dataset):           # dataset -> Name of the dataset, i.e. 'soc-sign-bitcoinalpha-dataset' 
  import pandas as pd
  
  nodeTable = pd.read_csv('./iTrustBD/BTC-Alpha/'+dataset+'-nodes-table.csv')
  edgeTable = pd.read_csv('./iTrustBD/BTC-Alpha/'+dataset+'-edge-table.csv')
  nodeTable['behavior'] = nodeTable['Class']

  edgeTableTargetSource = edgeTable[['Target', 'Source']].astype(str)
  neighborTableIn = edgeTableTargetSource.groupby(['Target'], as_index=False).agg(lambda col: ', '.join(col))
  neighborTableIn.columns = neighborTableIn.columns.str.replace('Source', 'Neighbors')

  neighborTableOut = edgeTableTargetSource.groupby(['Source'], as_index=False).agg(lambda col: ', '.join(col))
  neighborTableOut.columns = neighborTableOut.columns.str.replace('Target', 'Neighbors')

  return nodeTable, edgeTable, neighborTableIn, neighborTableOut

# Generate Random Probability

In [ ]:
def randProbability(min, max):
  # generate random floating point values
  import random
  return min + (random.random() * (max - min))

# Setting of Parameters

In [ ]:
def setParameters():
  beta_T = 0.5
  beta_S = beta_T

  meu_T = 0.25
  meu_S = meu_T

  return beta_T, beta_S, meu_T, meu_S

# Calculating Propotion of Neighbors

In [ ]:
def neighborProportion(nodeId): #nodeId, nodeTable, neighborTableIn, neighborTableOut
  setNeighbor = set(list(map(int, neighborTableIn.loc[neighborTableIn['Target'].astype(int) == int(nodeId)]['Neighbors'].item().split(","))))
  setNeighbor = set(list(map(int, neighborTableOut.loc[neighborTableOut['Source'].astype(int) == int(nodeId)]['Neighbors'].item().split(",")))).union(setNeighbor)

  countTrusty = 0
  countSuspecious = 0
  countNormal = 0

  for neighbor in setNeighbor:
    if int(nodeTable.loc[nodeTable['Id'].astype(int) == int(neighbor)].behavior.to_string(index=False)) == 0:
      countNormal = countNormal + 1
    elif int(nodeTable.loc[nodeTable['Id'].astype(int) == int(neighbor)].behavior.to_string(index=False)) == 1:
      countTrusty = countTrusty + 1
    else:
      countSuspecious = countSuspecious + 1
  
  return countTrusty/(countTrusty+countNormal+countSuspecious), countNormal/(countTrusty+countNormal+countSuspecious), countSuspecious/(countTrusty+countNormal+countSuspecious)
  #return sigma_T, sigma_N, sigma_S

# Susceptible-Infected-Susceptible (SIS) Model

In [ ]:
def modelSIS(epoch): #nodeTable, neighborTableIn, neighborTableOut
  import pandas as pd
  beta_T, beta_S, meu_T, meu_S = setParameters()
  for index in nodeTable.index:
    try:
      sigma_T, sigma_N, sigma_S = neighborProportion(nodeTable.Id[index])
      if(nodeTable.behavior[index].astype(int) == 0):
        P_NT = beta_T*sigma_T
        P_NS = beta_S*sigma_S
        if((P_NT >= P_NS) and (randProbability(0, 1) > (1 - P_NT))):
          nodeTable.at[index, 'behavior'] = 1 #nodeTable.behavior[index] = 1
        elif((P_NT < P_NS) and (randProbability(0, 1) > (1 - P_NS))):
          nodeTable.at[index, 'behavior'] = -1 #nodeTable.behavior[index] = -1
      elif((nodeTable.behavior[index].astype(int) == 1) and (randProbability(0, 1) > (1 - meu_T))):
        nodeTable.at[index, 'behavior'] = 0 #nodeTable.behavior[index] = 0
      elif((nodeTable.behavior[index].astype(int) == -1) and (randProbability(0, 1) > (1 - meu_S))):
        nodeTable.at[index, 'behavior'] = 0 #nodeTable.behavior[index] = 0
    except:
      pass
    filepath = '/content/drive/My Drive/PG/CSE6413 Network Science/Project Works/Datasets/Epoch_Out/node_table_SIS_{}.csv'.format(epoch)
    nodeTable.to_csv(filepath, index=False)
  #return nodeTable

# Behavior Spread

In [ ]:
def behaviorSpread(dataset, total_epoch):
  import pandas as pd
  import random
  global nodeTable, edgeTable, neighborTableIn, neighborTableOut
  nodeTable, edgeTable, neighborTableIn, neighborTableOut = setDatasets(dataset)
  filepath = '/content/drive/My Drive/PG/CSE6413 Network Science/Project Works/Datasets/Epoch_Out/node_table_SIS_{}.csv'.format(0)
  nodeTable.to_csv(filepath, index=False)
  epoch = 0
  while epoch < total_epoch:
    epoch = epoch + 1
    try:
      modelSIS(epoch)
    except:
      pass

In [ ]:
dataset = 'soc-sign-bitcoinalpha-dataset'
behaviorSpread(dataset, total_epoch=100)